Домашняя работа
по теме «Метапрограммирование в Python»

Задача 1 Применение метаклассов

In [1]:
# Метакласс
class AttrLoggingMeta(type):
    
    def __new__(cls, name, bases, attrs):
        new_class = super().__new__(cls, name, bases, attrs) # Создаем новый класс
                              
        # Переопределяем методы   __getattribute__, __setattr__
        original_getattribute = new_class.__getattribute__
        original_setattr = new_class.__setattr__

        # Метод логирования вызова
        def log_access(name, value): 
            print(f"Calling method '{name}'")

        # Метод логирования чтения
        def log_read(name, value, instance):
            print(f"Reading attribute '{name}'")
            
        # Метод логирования записи
        def log_write(name, value, instance):
            print(f"Writing attribute '{name}' value {value}")
                       
        def new_getattribute(self, name):
            value = original_getattribute(self, name)
            
            if callable(value): # Если метод                
                log_access(name, value) # Логируем вызов
            else:
                log_read(name, value, self) # иначе (атрибут) логируем чтение
            return value

        def new_setattr(self, name, value):
            log_write(name, value, self)
            original_setattr(self, name, value)

        # Заменяем методы
        new_class.__getattribute__ = new_getattribute
        new_class.__setattr__ = new_setattr
        
        return new_class

In [2]:
# Класс использующий метакласс AttrLoggingMeta
class LoggedClass(metaclass=AttrLoggingMeta):
    custom_method = 42 
    
    def other_custom_method(value):
        pass

In [3]:

instance = LoggedClass()
print(instance.custom_method)  # Чтение атрибута
instance.custom_method = 78    # Запись атрибута
instance.other_custom_method() # Вызов метода

Reading attribute 'custom_method'
42
Writing attribute 'custom_method' value 78
Calling method 'other_custom_method'


Задача 2 Динамическое создание класса

In [1]:
# Функция для динамического создания классов
def create_class_with_methods(class_name, attributes, methods):
    new_class = type(class_name, (object,), {**attributes, **methods})  # Создаем класс с помощью type
    return new_class # Возвращаем созданный класс

In [2]:
# Тест
attributes = { 'species': 'Human', 'age': 25 } 
methods = { 'greet': lambda self: f"Hello, I am a {self.species} and I am {self.age} years old." } 
DynamicClass = create_class_with_methods('DynamicClass', attributes, methods) 
instance = DynamicClass() 
print(instance.greet())

Hello, I am a Human and I am 25 years old.


Задача 3 Генерация кода

In [4]:
# Функция для динамической генерации функций
def generate_complex_function(func_name, param_names, body):
    params = ', '.join(param_names) # Формируем строку с определением функции
    function_code = f"def {func_name}({params}):\n" + '  ' + body # Формируем код функции
    print(function_code) # Печать кода функции для проверки
    
    # Создаем глобальный и локальный контекст для выполнения кода
    global_context = {}
    local_context = {}
    
    exec(function_code, global_context, local_context) # Выполняем exec
        
    return local_context[func_name]# Возвращаем сгенерированную функцию

In [8]:
# Тест
function_name = 'complex_function' 
parameters = ['x', 'y'] 
function_body = """
    if x > y: 
        return x - y 
    else: 
        return y - x
"""
complex_func = generate_complex_function(function_name, parameters, function_body) 
print(f' result_1 = {complex_func(10, 5)} ')
print(f' result_2 = {complex_func(5, 10)} ')

def complex_function(x, y):
  
    if x > y: 
        return x - y 
    else: 
        return y - x

 result_1 = 5 
 result_2 = 5 
